In [1]:
import mmh3

In [2]:
import functools

In [3]:
import iteration_utilities

In [4]:
import random

In [5]:
import numpy as np

In [6]:
import math

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.python import debug as tf_debug

In [9]:
from enum import IntEnum

In [10]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [11]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
number_of_classes = np.max(testing_data[1]).__int__() + 1
number_of_input_channels = 1

In [13]:
number_of_hidden_layers = 100

In [14]:
number_of_computing_layers = number_of_hidden_layers + number_of_classes

In [15]:
number_of_layers = number_of_input_channels + number_of_computing_layers

In [16]:
preferred_number_samples_per_layer = number_of_layers # All the previous layers 

In [17]:
paddings = (0, 0), (2, 2), (2, 2)

In [18]:
random_seed = number_of_hidden_layers

In [19]:
weight_scale = 1.0

In [20]:
dense_weight_size = (number_of_layers + number_of_input_channels) * (number_of_layers - number_of_input_channels) // 2
print(dense_weight_size)

6160


In [21]:
def grid_sample(images, offset_x, offset_y):
    tf.assert_rank(images, 3)
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        indexing='ij',
    )
    
    tf.assert_equal(tf.shape(image_index), image_shape)
    tf.assert_equal(tf.shape(x_index), image_shape)
    tf.assert_equal(tf.shape(y_index), image_shape)

    
    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_equal(tf.shape(left_index), image_shape)
    tf.assert_equal(tf.shape(top_index), image_shape)
    tf.assert_equal(tf.shape(right_index), image_shape)
    tf.assert_equal(tf.shape(bottom_index), image_shape)    
    
    tf.assert_rank(left_index, 3)
    tf.assert_rank(right_index, 3)
    tf.assert_rank(bottom_index, 3)
    tf.assert_rank(top_index, 3)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index), axis=3))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index), axis=3))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index), axis=3))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index), axis=3))

    tf.assert_rank(images_top_left, 3)
    tf.assert_rank(images_top_right, 3)
    tf.assert_rank(images_bottom_left, 3)
    tf.assert_rank(images_bottom_right, 3)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 3)
    return output

In [22]:
def model_fn(features, labels, mode, params, config):
    random.seed(number_of_hidden_layers)
    padded_input = tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings)

    score_weight = tf.get_variable(
        name="score_weight",
        initializer=tf.random_normal_initializer(),
        dtype=tf.float32,
        shape=(dense_weight_size, )
    )
    offset_x_weight = tf.get_variable(
        name="offset_x_weight",
        initializer=tf.random_normal_initializer(),
        dtype=tf.float32,
        shape=(dense_weight_size, )
    )
    offset_y_weight = tf.get_variable(
        name="offset_y_weight",
        initializer=tf.random_normal_initializer(),
        dtype=tf.float32,
        shape=(dense_weight_size, )
    )
    score_bias = tf.get_variable(
        name="score_bias",
        initializer=tf.zeros_initializer(),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    offset_x_bias = tf.get_variable(
        name="offset_x_bias",
        initializer=tf.random_uniform_initializer(minval=-3.0, maxval=3.0),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    offset_y_bias = tf.get_variable(
        name="offset_y_bias",
        initializer=tf.random_uniform_initializer(minval=-3.0, maxval=3.0),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    score_scale = tf.get_variable(
        name="score_scale",
        initializer=tf.sqrt(weight_scale / (tf.range(number_of_layers, dtype=tf.float32) + tf.constant(1e-10))),
        dtype=tf.float32
    )
    offset_x_scale = tf.get_variable(
        name="offset_x_scale",
        initializer=tf.sqrt(weight_scale / (tf.range(number_of_layers, dtype=tf.float32) + tf.constant(1e-10))),
        dtype=tf.float32
    )
    offset_y_scale = tf.get_variable(
        name="offset_y_scale",
        initializer=tf.sqrt(weight_scale / (tf.range(number_of_layers, dtype=tf.float32) + tf.constant(1e-10))),
        dtype=tf.float32
    )
    

    def score_map(previous_layers, number_of_previous_layers, next_layer_id):
        weight_start = (number_of_input_channels + next_layer_id) * (next_layer_id - number_of_input_channels) // 2
        weight_end = weight_start + number_of_previous_layers
        tf.assert_rank(weight_start, 0)
        
        # TODO test if the performance improves when using SparseTensor
        tf.assert_rank(previous_layers, 4)

        def indexed_sum(weight, bias, scale):
            tf.assert_rank(weight, 1)
            weight_slice = weight[weight_start:weight_end] * scale[next_layer_id]

            tf.assert_equal(tf.shape(weight_slice)[0], tf.shape(previous_layers)[3])
            return tf.tensordot(
                previous_layers,
                weight_slice,
                axes=1
            ) + bias[next_layer_id]

        return grid_sample(
            indexed_sum(score_weight, score_bias, score_scale),
            indexed_sum(offset_x_weight, offset_x_bias, offset_x_scale),
            indexed_sum(offset_y_weight, offset_y_bias, offset_y_scale),
        )
    
    feature_layers, i = tf.while_loop(
        lambda layers, i: i < number_of_input_channels + number_of_hidden_layers,
        lambda layers, i: (
            tf.concat(
                (
                    layers,
                    tf.expand_dims(
                        score_map(
                            layers,
                            number_of_previous_layers=i,
                            next_layer_id=i
                        ),
                        axis=3,
                    )
                ),
                axis=3,
            ),
            i + 1
        ),
        (
            tf.expand_dims(padded_input, axis=3),
            tf.constant(number_of_input_channels),
        ),
        shape_invariants=(
            tf.TensorShape((padded_input.shape[0], padded_input.shape[1], padded_input.shape[2], None)), 
            tf.TensorShape(())
        )
    )
    tf.assert_equal(i, tf.cast(number_of_input_channels + number_of_hidden_layers, tf.int32))

    layers, i = tf.while_loop(
        lambda layers, i: i < number_of_layers,
        lambda layers, i: (
            tf.concat(
                (
                    layers,
                    tf.expand_dims(
                        score_map(
                            feature_layers,
                            number_of_previous_layers=number_of_input_channels + number_of_hidden_layers,
                            next_layer_id=i
                        ),
                        axis=3,
                    )
                ),
                axis=3,
            ),
            i + 1
        ),
        (feature_layers, number_of_input_channels + number_of_hidden_layers),
        shape_invariants=(
            tf.TensorShape((padded_input.shape[0], padded_input.shape[1], padded_input.shape[2], None)), 
            tf.TensorShape(())
        )
    )

    tf.assert_equal(i, tf.cast(number_of_layers, tf.int32))

    scores = layers[:, padded_input.shape[1] // 2, padded_input.shape[2] // 2, number_of_layers - number_of_classes:number_of_layers]
    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    if labels is None:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
        )
    else:
        loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
        }
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=loss,
            train_op=optimizer.minimize(loss, global_step=tf.train.get_global_step()),
            eval_metric_ops=eval_metric_ops,
        )

In [23]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/offnet{number_of_hidden_layers}",
    session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            allow_growth=True,
        ),
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            ),
        )
    )
)

In [24]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/offnet100', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f513284b470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
batch_size = 32

In [26]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).batch(batch_size)

In [27]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [28]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [29]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/offnet100/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into models/offnet100/model.ckpt.
INFO:tensorflow:loss = 2.2882938, step = 1
INFO:tensorflow:global_step/sec: 2.62954
INFO:tensorflow:loss = 1.0029163, step = 101 (38.030 sec)
INFO:tensorflow:global_step/sec: 2.70992
INFO:tensorflow:loss = 0.65639234, step = 201 (36.901 sec)
INFO:tensorflow:global_step/sec: 2.69892
INFO:tensorflow:loss = 0.68923116, step = 301 (37.052 sec)
INFO:tensorflow:global_step/sec: 2.64078
INFO:tensorflow:loss

KeyboardInterrupt: 

In [ ]:
# estimator.evaluate(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1))

In [ ]:
# tuple(estimator.predict(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1)))

In [ ]:
# estimator.train(input_fn=training_dataset, steps=1)